In [1]:
# !conda install -c conda-forge chardet 
# !pip install git+https://github.com/MaartenGr/BERTopic.git#egg=bertopic
# !python -m pip install charset-normalizer==2.1.0

  Cloning https://github.com/MaartenGr/BERTopic.git to c:\users\filipp03\appdata\local\temp\1\pip-install-4zn3kmoa\bertopic_2f8f3a61ffb94f95b2d948e024daeee1
  Resolved https://github.com/MaartenGr/BERTopic.git to commit fca5a4f9df149609c7e3458d6b2c421194cea62c
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/MaartenGr/BERTopic.git 'C:\Users\filipp03\AppData\Local\Temp\1\pip-install-4zn3kmoa\bertopic_2f8f3a61ffb94f95b2d948e024daeee1'


In [2]:
from bertopic import BERTopic
# from sentence_transformers import SentenceTransformer
from pull_graph_data_constellation_2023 import (
    driver,
    get_all_study_names,
    grab_all_doc_content_ordered,
)
import pandas as pd

In [3]:
# create graphdb session
session = driver.session()

topic_model = BERTopic(embedding_model="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
# "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
# "emilyalsentzer/Bio_ClinicalBERT"

In [4]:
def split_string(string, n):
    return [string[i:i+n] for i in range(0, len(string), n)]

# query graph for document data
study_ids = get_all_study_names(session)[:3]

docs = []  # holds reconstructed protocols for each study has strings
for study_id in study_ids:
    # gather document chunks from graph
    doc_chunks_split = []
    doc_chunks = grab_all_doc_content_ordered(study_id, session)
    
    for chunk in doc_chunks:
        if len(chunk) > 1000:
            split_chunks = split_string(chunk, 1000)
            doc_chunks_split.extend(split_chunks)
        else:
            doc_chunks_split.append(chunk)
    docs.extend(doc_chunks_split)
    


In [5]:
len(docs)

1510

In [6]:
topics, probs = topic_model.fit_transform(docs)

# Each row in the df corresponds to a topic (or a cluster)
# 'Count' is how many documents are clustered in it
# The integer is the index of the topic
df = topic_model.get_topic_info()
print(df)

# Write DataFrame to a CSV file
df.to_csv("PubMEDBERT_topics.csv", index=False)

No sentence-transformers model found with name C:\Users\filipp03/.cache\torch\sentence_transformers\microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\filipp03/.cache\torch\sentence_transformers\microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Ber

    Topic  Count                                             Name  \
0      -1    382                                -1_the_and_of_for   
1       0    120                    0_the_study_investigator_will   
2       1     73                               1_td_tr_colspan_10   
3       2     72                 2_bempegaldesleukin_dose_be_nktr   
4       3     57                               3_pd_with_in_cells   
5       4     43                            4_td_tr_table_section   
6       5     43  5_considerations_statistical_design_operational   
7       6     42              6_disease_imaging_progression_tumor   
8       7     35                       7_dose_dlt_combination_the   
9       8     32                8_cancer_patients_in_chemotherapy   
10      9     31               9_itch_itching_items_questionnaire   
11     10     31            10_topical_eg_medications_concomitant   
12     11     30                     11_will_the_primary_analysis   
13     12     30                  

In [ ]:
# Each topic has a number of top keywords associated with it, some are show under the 'Name' column
# To get more keywords under a topic: 
topic_model.get_topic(50)

In [ ]:
# To get the top documents associated (clustered the cloesest) with a given topic: 
topic_model.get_representative_docs(42)

In [ ]:
# To build a dataframe assigning each document with its topic:
df = pd.DataFrame({"topics":topics, "Documents": docs})
df

In [ ]:
topic_model.visualize_topics()

In [ ]:
# close graphdb session at end
session.close()
driver.close()